In [ ]:
import pystan

In [ ]:
X = pd.read_csv('./variables/devision_rosenID.csv', index_col=0)
y = pd.read_csv('./variables/target.csv', index_col=0 ,names=['飲食店数'])

In [ ]:
def zscore(x, axis = None):
    xmean = x.mean(axis=axis)
    xstd  = np.std(x, axis=axis)
    zscore = (x-xmean)/xstd
    return zscore

In [ ]:
X.iloc[:, :-1] = zscore(X.iloc[:, :-1], axis=0)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [ ]:
rosen = dict(zip(X['路線ID'].unique(), range(1, 1+len(X['路線ID'].unique()))))
Rosen_train = []
for i in range(len(X_train)):
    Rosen_train.append(rosen[X_train['路線ID'][i]])
Rosen_test = []
for i in range(len(X_test)):
    Rosen_test.append(rosen[X_test['路線ID'][i]])

In [ ]:
X_train = X_train.drop('路線ID', axis=1)
X_test = X_test.drop('路線ID', axis=1)

In [ ]:
data = dict(
    N_train=X_train.shape[0],
    N_test=X_test.shape[0],
    N_explanatory=X_train.shape[1],
    N_rosen=len(rosen),
    X_train=X_train.values,
    X_test=X_test.values,
    Rosen_train=Rosen_train,
    Rosen_test=Rosen_test,
    Y_train=y_train['飲食店数'].values,
    Y_test=y_test['飲食店数'].values
)

In [ ]:
model = '''
data {
    int N_train;
    int N_test;
    int N_explanatory;
    int N_rosen;
    matrix[N_train, N_explanatory] X_train;
    matrix[N_test, N_explanatory] X_test;
    int<lower=1> Rosen_train[N_train];
    int<lower=1> Rosen_test[N_test];
    vector<lower=0>[N_train] Y_train;
    vector<lower=0>[N_test] Y_test;
}

parameters {
    real mu_a;
    real mu_b[N_explanatory];
    real<lower=0> mu_s;
    real<lower=0> s_a;
    real<lower=0> s_b[N_explanatory];
    real<lower=0> s_s;
    vector[N_rosen] a;
    matrix[N_explanatory, N_rosen] b;
    vector<lower=0>[N_rosen] s;
}

model {
    a ~ student_t(4, mu_a, s_a);
    for (i in 1:N_explanatory)
        b[i,] ~ student_t(4, mu_b[i], s_b[i]);
    s ~ student_t(4, mu_s, s_s);
    for (i in 1:N_train)
        Y_train[i] ~ student_t(4, a[Rosen_train[i]] + X_train[i,]*b[,Rosen_train[i]], s[Rosen_train[i]]);
}

generated quantities{
    vector[N_test] predict;
    for (i in 1:N_test)
        predict[i] = student_t_rng(4, a[Rosen_test[i]] + X_test[i,]*b[,Rosen_test[i]], s[Rosen_test[i]]);
}
'''

In [ ]:
fit = pystan.stan(model_code=model, data=data, chains=3, iter=500, warmup=100, thin=1)

In [ ]:
fit

In [ ]:
#MCMCサンプリングの結果を抽出
ms = fit.extract(permuted=False, inc_warmup=True)
#ウォームアップ（バーンイン）のサイズを取得
iter_from = fit.sim['warmup']
#ウォームアップの区間を省く
iter_range = np.arange(iter_from, ms.shape[0])
#各変数名を取得
paraname = fit.sim['fnames_oi']

#※※※今回は全て描画したいので、こちらを使う
iter_start = np.arange(0, ms.shape[0])

In [ ]:
#seabornのcolorpalette
palette = sns.color_palette()
#おまじない？
sns.set(font_scale=1)
sns.set_style("ticks")
sns.despine(offset=10, trim=True)

fig,axes  = plt.subplots(nrows=4, ncols=3, figsize=(15,10))

for i in range(4):
    for j in range(3):
        axes[i,j].plot(iter_start, ms[iter_start, :, i*3+j])
        axes[i,j].set_title(paraname[i*3+j])
        axes[i,j].set_xlabel('mcmc_size')
        axes[i,j].set_ylabel('parameter')
        axes[i,j].grid(True)

fig.show()

In [ ]:
summary = pd.DataFrame(data=fit.summary()['summary'], index=fit.summary()['summary_rownames'], columns=fit.summary()['summary_colnames'])

In [ ]:
summary

In [ ]:
pred = summary.query('index.str.startswith("predict")', engine='python')['50%'].values

In [ ]:
#RMSE
np.sqrt(np.mean((y_test['飲食店数'].values-pred)**2))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test.values, pred)

大量の特徴量を使ってlightgbmでやると、0.95870503227042

In [ ]:
fig, ax = plt.subplots()
ax.scatter(pred, y_test['飲食店数'], edgecolors=(0, 0, 0))
ax.plot([y_test['飲食店数'].min(), y_test['飲食店数'].max()], [y_test['飲食店数'].min(), y_test['飲食店数'].max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()

In [ ]:
def plot_actual_predicted(actual, predicted):
    tmp = pd.DataFrame({'actual': actual, 'predicted': predicted}).sort_values(['actual'])
    plt.scatter(range(tmp.shape[0]), tmp['predicted'], color='green')
    plt.scatter(range(tmp.shape[0]), tmp['actual'], color='blue')
    plt.show()
    del tmp

plot_actual_predicted(y_test['飲食店数'], pred)